In [ ]:
import pandas as pd
import json

In [ ]:
path = "/home/tuyendv/Desktop/reranker/data/raw/vi_data.json"
df = pd.read_json(path)
df.head()

In [ ]:
data = []
for index in df.index:
    query = df["question"][index]
    positive = df["positive_sample"][index]
    negative_samples = df["negative_samples"][index]
    
    passages = []
    
    passages.append(
        {
            "passage_text":positive,
            "is_selected":1
        }
    )
    
    for negative in negative_samples:
        passages.append(
            {
                "passage_text":negative,
                "is_selected":0
            }
        )
        
    data.append(
        {
            "query":query,
            "passages":passages
        }
    )

In [ ]:
def save_data(path, data):
    with open(path, "w", encoding="utf-8") as f:
        for sample in data:
            json_obj = json.dumps(sample, ensure_ascii=False)
            f.write(json_obj+"\n")

path="/home/tuyendv/Desktop/reranker/data/ir/vi_data.json"
save_data(path, data)

### v1

In [ ]:
import json

In [ ]:
path = "/home/tuyendv/Desktop/reranker/data/raw/test_hidden_v1.1.json"

data = []
with open(path, "r", encoding="utf-8") as f:
    for line in f.readlines():
        line = json.loads(line.strip())
        
        passages = [
            {
                "passage_text":i["passage_text"],
                "is_selected":i["is_selected"]
                } for i in line["passages"]
            ]
        data.append(
            {
                "query":line["query"],
                "passages":passages
            }
        )

In [ ]:
def save_data(path, data):
    with open(path, "w", encoding="utf-8") as f:
        for sample in data:
            json_obj = json.dumps(sample, ensure_ascii=False)
            f.write(json_obj+"\n")

path="/home/tuyendv/Desktop/reranker/data/ir/test_hidden_v1.1.json"
save_data(path, data)

### temp

In [ ]:
import os 
from glob import glob
import json
import pandas as pd

In [ ]:
def check(sample):
    count=0 
    for passage in sample["passages"]:
        if passage["is_selected"] == 1:
            count+=1

    if count == 0 or count > 1:
        return False
    return True

def load_file(path):
    with open(path, "r", encoding="utf-8") as f:
        data = []
        for line in f.readlines():
            json_obj = json.loads(line.strip())
            if check(json_obj):
                data.append(json_obj)
            
    return data

def load_data(path):
    data = []
    for _file in glob(path+"/*.json"):
        data += load_file(_file)
            
    return data

path = "/home/tuyendv/Desktop/reranker/data/ir"
data = load_data(path)
len(data)

In [ ]:
data= pd.DataFrame(data)
data.head()

In [ ]:
def get_length(query, passages):
    query_length = len(query.split())
    
    lengths = []
    for pa in passages:
        tmp = pa["passage_text"]
        
        lengths.append(len(tmp.split()) + query_length)
        
    return lengths

data["length"] = data.apply(lambda x: get_length(x["query"], x["passages"]), axis=1)

In [ ]:
lengths = [i for j in data.length.tolist() for i in j]

In [ ]:
pd.DataFrame(lengths, columns=["lengths"]).lengths.hist(bins=100)